In [2]:
from wand.image import Image
 
# 将pdf文件转为jpg图片文件
# ./PDF_FILE_NAME 为pdf文件路径和名称
fn="//home//jack//文档//大智小超//乐高教育.pdf"

image_pdf = Image(filename=fn,resolution=300)
image_jpeg = image_pdf.convert('jpg')
 
# wand已经将PDF中所有的独立页面都转成了独立的二进制图像对象。我们可以遍历这个大对象，并把它们加入到req_image序列中去。
req_image = []
for img in image_jpeg.sequence:
    img_page = Image(image=img)
    req_image.append(img_page.make_blob('jpg'))
    
print(req_image)
 
# 遍历req_image,保存为图片文件
# i = 0
# for img in req_image:
#     ff = open(str(i)+'.jpg','wb')
#     ff.write(img)
#     ff.close()
#     i += 1
    
    
    

PolicyError: not authorized `//home//jack//文档//大智小超//乐高教育.pdf' @ error/constitute.c/ReadImage/412

In [3]:
import os
inmport tqdm
from pdf2image import convert_from_path,convert_from_bytes
import tempfile
from pdf2image.exceptions import (
    PDFInfoNotInstalledError,
    PDFPageCountError,
    PDFSyntaxError
)
 
def pdf2image2(pdfPath, imagePath, pageNum):
    with tempfile.TemporaryDirectory() as path:
            images_from_path = convert_from_path(pdfPath, output_folder=path, dpi=96)
            print(len(images_from_path))
#             for image in images_from_path:
#                 if not os.path.exists(imagePath):
#                     os.makedirs(imagePath)
#                 image.save(imagePath+'/'+'psReport_%s.png' % images_from_path.index(image), 'PNG')
#             print(images_from_path)
        
if __name__=="__main__":
    pdf='//home//jack//文档//大智小超//乐高教育.pdf'
    jpg='//home//jack//文档//大智小超'
    pagenum=2
    pdf2image2(pdf,jpg,pagenum)

219
